In [1]:
import pandas as pd
from functools import reduce
import numpy as np

from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, SVR
from sklearn.neural_network import MLPClassifier

In [2]:
feature_dir = 'data/features/'

In [3]:
df_twitter = pd.read_csv(feature_dir+'twitter_features.csv')
df_news = pd.read_csv(feature_dir+'news_features.csv')
df_out= pd.read_csv(feature_dir+'index_features.csv')

In [4]:
clf = SVC(C=1.0)
clf2 = SVR(C=1.0)

In [5]:
df_y = df_out
df_X = pd.merge(df_twitter, df_news, on='Date', how='outer')

In [6]:
df_y

,Date,Change VIX,Direction VIX,Lag 1 Change VIX,Lag 1 Direction VIX,Lag 2 Change VIX,Lag 2 Direction VIX,Lag 2 Significant VIX,Lag 5 Change VIX,Lag 5 Direction VIX,Change SPY,Direction SPY,Lag 1 Change SPY,Lag 1 Direction SPY,Lag 2 Change SPY,Lag 2 Direction SPY,Lag 2 Significant SPY,Lag 5 Change SPY,Lag 5 Direction SPY
0,2015-01-02,0.116279,1,0.080518,1,0.144707,1,1,0.144707,1,-0.007741,-1,-0.010708,-1,-0.020787,-1,-1,-0.020787,-1
1,2015-01-05,0.080518,1,0.059406,1,0.050026,1,1,0.050026,1,-0.010708,-1,-0.010188,-1,-0.013469,-1,-1,-0.013469,-1
2,2015-01-06,0.059406,1,-0.008854,-1,-0.118052,-1,-1,-0.118052,-1,-0.010188,-1,-0.003315,-1,0.009501,1,0,0.009501,1
3,2015-01-07,-0.008854,-1,-0.110174,-1,-0.184119,-1,-1,-0.184119,-1,-0.003315,-1,0.012859,1,0.024724,1,1,0.024724,1
4,2015-01-08,-0.110174,-1,-0.083101,-1,0.005020,1,0,0.005020,1,0.012859,1,0.011715,1,0.001961,1,0,0.001961,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083,2019-04-24,-0.010269,-1,0.060654,1,0.072626,1,1,0.072626,1,0.007259,1,-0.002288,-1,-0.002357,-1,0,-0.002357,-1
1084,2019-04-25,0.060654,1,0.011287,1,-0.018811,-1,-1,-0.018811,-1,-0.002288,-1,-0.000068,-1,0.004758,1,0,0.004758,1
1085,2019-04-26,0.011287,1,-0.029762,-1,-0.024554,-1,-1,-0.024554,-1,-0.000068,-1,0.004827,1,0.004759,1,0,0.004759,1
1086,2019-04-29,-0.029762,-1,0.005368,1,-0.013804,-1,-1,-0.013804,-1,0.004827,1,-0.000068,-1,0.004123,1,0,0.004123,1


In [7]:
df_X

,Date,BarackObama_sentiment_score,BarackObama_heuristic_score,cnnbrk_sentiment_score,cnnbrk_heuristic_score,KimKardashian_sentiment_score,KimKardashian_heuristic_score,Trump_sentiment_score,Trump_heuristic_score,New York Times_sentiment_score,New York Times_polar_score,CNN_sentiment_score,CNN_polar_score,Washington Post_sentiment_score,Washington Post_polar_score,NYT_sentiment_score,NYT_polar_score
0,2015-01-02,-0.107225,-0.114245,NaN,NaN,NaN,NaN,0.323554,0.331358,NaN,NaN,-0.176336,-0.1492,NaN,NaN,-0.012855,-0.0970
1,2015-01-05,0.610250,0.628957,NaN,NaN,NaN,NaN,0.352101,0.383644,NaN,NaN,-0.281860,-0.8519,NaN,NaN,0.021248,0.0937
2,2015-01-06,0.160900,0.549501,NaN,NaN,0.193586,0.403680,0.354565,0.295516,NaN,NaN,0.100810,0.1651,NaN,NaN,-0.018193,-0.1756
3,2015-01-07,0.469467,0.445845,NaN,NaN,0.183080,0.247557,0.109130,-0.421761,NaN,NaN,-0.229275,-0.4767,NaN,NaN,-0.045886,0.1026
4,2015-01-08,0.370700,0.621998,NaN,NaN,0.571667,0.673277,0.099112,-0.122951,NaN,NaN,-0.360086,-0.7003,NaN,NaN,-0.034352,-0.0730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083,2019-04-24,NaN,NaN,NaN,NaN,NaN,NaN,0.252729,0.244343,NaN,NaN,NaN,NaN,NaN,NaN,-0.050849,-0.1125
1084,2019-04-25,NaN,NaN,NaN,NaN,NaN,NaN,-0.215995,0.026744,NaN,NaN,NaN,NaN,NaN,NaN,-0.028734,0.0418
1085,2019-04-26,NaN,NaN,NaN,NaN,NaN,NaN,0.192100,0.323469,NaN,NaN,NaN,NaN,NaN,NaN,-0.078153,-0.2229
1086,2019-04-29,NaN,NaN,NaN,NaN,NaN,NaN,0.354891,0.354788,NaN,NaN,NaN,NaN,NaN,NaN,-0.126724,-0.1281


In [8]:
df_X.columns

Index(['Date', 'BarackObama_sentiment_score', 'BarackObama_heuristic_score',
       'cnnbrk_sentiment_score', 'cnnbrk_heuristic_score',
       'KimKardashian_sentiment_score', 'KimKardashian_heuristic_score',
       'Trump_sentiment_score', 'Trump_heuristic_score',
       'New York Times_sentiment_score', 'New York Times_polar_score',
       'CNN_sentiment_score', 'CNN_polar_score',
       'Washington Post_sentiment_score', 'Washington Post_polar_score',
       'NYT_sentiment_score', 'NYT_polar_score'],
      dtype='object')

In [9]:
X = df_X.reset_index(drop=True)[['NYT_sentiment_score', 'Trump_sentiment_score']]
y = df_y.reset_index(drop=True)
# X = X.drop(columns=['Date'])
X.fillna(0, inplace=True)
y = y['Lag 2 Significant VIX']
# y.fillna(y.mean(), inplace=True)

In [10]:
np.isinf(X).all()

NYT_sentiment_score      False
Trump_sentiment_score    False
dtype: bool

In [11]:
X

,NYT_sentiment_score,Trump_sentiment_score
0,-0.012855,0.323554
1,0.021248,0.352101
2,-0.018193,0.354565
3,-0.045886,0.109130
4,-0.034352,0.099112
...,...,...
1083,-0.050849,0.252729
1084,-0.028734,-0.215995
1085,-0.078153,0.192100
1086,-0.126724,0.354891


In [12]:
y

0       1
1       1
2      -1
3      -1
4       0
       ..
1083    1
1084   -1
1085   -1
1086   -1
1087    1
Name: Lag 2 Significant VIX, Length: 1088, dtype: int64

In [13]:
y.value_counts()

-1    526
 1    455
 0    107
Name: Lag 2 Significant VIX, dtype: int64

In [14]:
# Guessing Most common class always
succ = y.value_counts().max()/(y.count())

In [15]:
succ

0.4834558823529412

In [16]:
np.where(np.isnan(X))

(array([], dtype=int32), array([], dtype=int32))

In [17]:
y

0       1
1       1
2      -1
3      -1
4       0
       ..
1083    1
1084   -1
1085   -1
1086   -1
1087    1
Name: Lag 2 Significant VIX, Length: 1088, dtype: int64

In [18]:
def run_experiment(model, X, y):
    scores = []
    kf = KFold(n_splits=5)
    for train_index, test_index in kf.split(X):
        X_train, X_test, y_train, y_test = X.loc[train_index],X.loc[test_index], y.loc[train_index], y.loc[test_index]
        X_train.fillna(X_train.mean(), inplace=True)
        model.fit(X_train, y_train)
        scores.append(model.score(X_test, y_test))
    print('Scores',scores)
    print('Average score', sum(scores)/len(scores))

run_experiment(clf, X, y)

Scores [0.4954128440366973, 0.46788990825688076, 0.44495412844036697, 0.4930875576036866, 0.5161290322580645]
Average score 0.4834946941191392


c:\users\rudyt\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\rudyt\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\rudyt\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning."

In [19]:
X = df_X.reset_index(drop=True).set_index('Date')[['NYT_sentiment_score']]#Trump_sentiment_score
y = df_y.reset_index(drop=True).set_index('Date')
X.fillna(0, inplace=True)
y = y['Lag 2 Significant VIX']

df_X.reset_index(drop=True).set_index('Date')

,BarackObama_sentiment_score,BarackObama_heuristic_score,cnnbrk_sentiment_score,cnnbrk_heuristic_score,KimKardashian_sentiment_score,KimKardashian_heuristic_score,Trump_sentiment_score,Trump_heuristic_score,New York Times_sentiment_score,New York Times_polar_score,CNN_sentiment_score,CNN_polar_score,Washington Post_sentiment_score,Washington Post_polar_score,NYT_sentiment_score,NYT_polar_score
Date,,,,,,,,,,,,,,,,
2015-01-02,-0.107225,-0.114245,NaN,NaN,NaN,NaN,0.323554,0.331358,NaN,NaN,-0.176336,-0.1492,NaN,NaN,-0.012855,-0.0970
2015-01-05,0.610250,0.628957,NaN,NaN,NaN,NaN,0.352101,0.383644,NaN,NaN,-0.281860,-0.8519,NaN,NaN,0.021248,0.0937
2015-01-06,0.160900,0.549501,NaN,NaN,0.193586,0.403680,0.354565,0.295516,NaN,NaN,0.100810,0.1651,NaN,NaN,-0.018193,-0.1756
2015-01-07,0.469467,0.445845,NaN,NaN,0.183080,0.247557,0.109130,-0.421761,NaN,NaN,-0.229275,-0.4767,NaN,NaN,-0.045886,0.1026
2015-01-08,0.370700,0.621998,NaN,NaN,0.571667,0.673277,0.099112,-0.122951,NaN,NaN,-0.360086,-0.7003,NaN,NaN,-0.034352,-0.0730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-04-24,NaN,NaN,NaN,NaN,NaN,NaN,0.252729,0.244343,NaN,NaN,NaN,NaN,NaN,NaN,-0.050849,-0.1125
2019-04-25,NaN,NaN,NaN,NaN,NaN,NaN,-0.215995,0.026744,NaN,NaN,NaN,NaN,NaN,NaN,-0.028734,0.0418
2019-04-26,NaN,NaN,NaN,NaN,NaN,NaN,0.192100,0.323469,NaN,NaN,NaN,NaN,NaN,NaN,-0.078153,-0.2229


In [35]:
import datetime
import calendar
from dateutil.relativedelta import relativedelta

def run_experiment2(model, X, y):
    scores = []
    
    train_start = datetime.date(2015, 1, 1)
    train_end = (train_start + relativedelta(months=2))
    days = calendar.monthrange(train_end.year, train_end.month)[1]
    train_end = train_end.replace(day=days)
    test_start = (train_start + relativedelta(months=3)).replace(day=1)
    test_end = (train_start + relativedelta(months=3)).replace(day=14)
    
    while train_end < datetime.date(2019, 4, 1):
        X_train = X.loc[str(train_start):str(train_end)]
        X_train = X_train.fillna(X_train.mean())
        y_train = y.loc[str(train_start):str(train_end)]
        
        X_test = X.loc[str(test_start):str(test_end)]
        y_test = y.loc[str(test_start):str(test_end)]
        
        model.fit(X_train, y_train)
        scores.append(model.score(X_test, y_test))
        
        train_start = (train_start + relativedelta(months=1))
        train_end = (train_start + relativedelta(months=2))
        days = calendar.monthrange(train_end.year, train_end.month)[1]
        train_end = train_end.replace(day=days)
        test_start = (train_start + relativedelta(months=3)).replace(day=1)
        test_end = (train_start + relativedelta(months=3)).replace(day=14)
    print([round(x, 5) for x in scores])
    print('Average score', sum(scores)/len(scores))
    print('Total runs:', len(scores))

run_experiment2(clf, X, y)

c:\users\rudyt\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\rudyt\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\rudyt\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning."

[0.66667, 0.5, 0.5, 0.55556, 0.5, 0.22222, 0.4, 0.3, 0.3, 0.22222, 0.2, 0.4, 0.5, 0.5, 0.5, 0.22222, 0.4, 0.55556, 0.3, 0.4, 0.3, 0.22222, 0.5, 0.3, 0.22222, 0.5, 0.3, 0.66667, 0.4, 0.66667, 0.3, 0.2, 0.5, 0.66667, 0.4, 0.3, 0.2, 0.7, 0.6, 0.66667, 0.6, 0.55556, 0.3, 0.4, 0.33333, 0.0, 0.4, 0.5, 0.5]
Average score 0.41519274376417237
Total runs: 49


c:\users\rudyt\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\rudyt\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\rudyt\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning."

In [21]:
def rfc_param_selection(X, y):
    n_estims= [200, 700]
    max_feats= ['auto', 'sqrt', 'log2']
    param_grid = {'n_estimators': n_estims, 'max_features' : max_feats}
    grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, verbose=5)
    grid_search.fit(X, y)
    grid_search.best_params_
    print(grid_search.best_estimator_)
    print(grid_search.best_score_)
    return grid_search.best_params_

In [22]:
def svc_param_selection(X, y):
    kernels = ['rbf','linear']
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas, 'kernel':kernels}
    grid_search = GridSearchCV(SVC(), param_grid, cv=5, verbose=5)
    grid_search.fit(X, y)
    grid_search.best_params_
    print(grid_search.best_estimator_)
    print(grid_search.best_score_)
    return grid_search.best_params_

In [23]:
def MLP_param_selection(X, y):
    solvers = ['lbfgs', 'sgd']
    max_iters = [1000,1500,2000 ]
    alphas = [10,100,1000,10000]
    hidden_layer_sizes = [10,12,15]
    random_states = [i for i in range(10)]
    param_grid = {'solver': solvers, 'max_iter': max_iters, 'alpha': alphas, 'hidden_layer_sizes':hidden_layer_sizes, 'random_state':random_states}
    

    grid_search = GridSearchCV(MLPClassifier(), param_grid, cv=5, verbose=5)
    grid_search.fit(X, y)
    grid_search.best_params_
    print(grid_search.best_estimator_)
    print(grid_search.best_score_)
    return grid_search.best_params_


In [24]:
svc_param_selection(X,y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV]  C=0.001, gamma=0.001, kernel=rbf, score=0.4840182648401826, total=   0.0s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV]  C=0.001, gamma=0.001, kernel=rbf, score=0.481651376146789, total=   0.0s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


[CV]  C=0.001, gamma=0.001, kernel=rbf, score=0.4838709677419355, total=   0.0s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV]  C=0.001, gamma=0.001, kernel=rbf, score=0.4838709677419355, total=   0.0s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV]  C=0.001, gamma=0.001, kernel=rbf, score=0.4838709677419355, total=   0.0s
[CV] C=0.001, gamma=0.001, kernel=linear .............................
[CV]  C=0.001, gamma=0.001, kernel=linear, score=0.4840182648401826, total=   0.0s
[CV] C=0.001, gamma=0.001, kernel=linear .............................
[CV]  C=0.001, gamma=0.001, kernel=linear, score=0.481651376146789, total=   0.0s
[CV] C=0.001, gamma=0.001, kernel=linear .............................
[CV]  C=0.001, gamma=0.001, kernel=linear, score=0.4838709677419355, total=   0.0s
[CV] C=0.001, gamma=0.001, kernel=linear .............................
[CV]  C=0.001, gamma=0.001, kernel=linear, score=0.4838709677419355, total=   0.0s
[CV

[CV]  C=0.01, gamma=0.01, kernel=linear, score=0.4838709677419355, total=   0.0s
[CV] C=0.01, gamma=0.1, kernel=rbf ...................................
[CV]  C=0.01, gamma=0.1, kernel=rbf, score=0.4840182648401826, total=   0.0s
[CV] C=0.01, gamma=0.1, kernel=rbf ...................................
[CV]  C=0.01, gamma=0.1, kernel=rbf, score=0.481651376146789, total=   0.0s
[CV] C=0.01, gamma=0.1, kernel=rbf ...................................
[CV]  C=0.01, gamma=0.1, kernel=rbf, score=0.4838709677419355, total=   0.0s
[CV] C=0.01, gamma=0.1, kernel=rbf ...................................
[CV]  C=0.01, gamma=0.1, kernel=rbf, score=0.4838709677419355, total=   0.0s
[CV] C=0.01, gamma=0.1, kernel=rbf ...................................
[CV]  C=0.01, gamma=0.1, kernel=rbf, score=0.4838709677419355, total=   0.0s
[CV] C=0.01, gamma=0.1, kernel=linear ................................
[CV]  C=0.01, gamma=0.1, kernel=linear, score=0.4840182648401826, total=   0.0s
[CV] C=0.01, gamma=0.1, kerne

[CV]  C=0.1, gamma=1, kernel=linear, score=0.4838709677419355, total=   0.0s
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV]  C=0.1, gamma=1, kernel=linear, score=0.4838709677419355, total=   0.0s
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV]  C=0.1, gamma=1, kernel=linear, score=0.4838709677419355, total=   0.0s
[CV] C=1, gamma=0.001, kernel=rbf ....................................
[CV]  C=1, gamma=0.001, kernel=rbf, score=0.4840182648401826, total=   0.0s
[CV] C=1, gamma=0.001, kernel=rbf ....................................
[CV]  C=1, gamma=0.001, kernel=rbf, score=0.481651376146789, total=   0.0s
[CV] C=1, gamma=0.001, kernel=rbf ....................................
[CV]  C=1, gamma=0.001, kernel=rbf, score=0.4838709677419355, total=   0.0s
[CV] C=1, gamma=0.001, kernel=rbf ....................................
[CV]  C=1, gamma=0.001, kernel=rbf, score=0.4838709677419355, total=   0.0s
[CV] C=1, gamma=0.001, kernel=rbf ......

[CV]  C=10, gamma=0.01, kernel=linear, score=0.4838709677419355, total=   0.0s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV]  C=10, gamma=0.01, kernel=linear, score=0.4838709677419355, total=   0.0s
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV]  C=10, gamma=0.1, kernel=rbf, score=0.4840182648401826, total=   0.0s
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV]  C=10, gamma=0.1, kernel=rbf, score=0.481651376146789, total=   0.0s
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV]  C=10, gamma=0.1, kernel=rbf, score=0.4838709677419355, total=   0.0s
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV]  C=10, gamma=0.1, kernel=rbf, score=0.4838709677419355, total=   0.0s
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV]  C=10, gamma=0.1, kernel=rbf, score=0.4838709677419355, total=   0.0s
[CV] C=10, gamma=0.1, kernel=linear ......

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    6.9s finished


{'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'}

In [25]:
rfc_param_selection(X,y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] max_features=auto, n_estimators=200 .............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_features=auto, n_estimators=200, score=0.3881278538812785, total=   0.5s
[CV] max_features=auto, n_estimators=200 .............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV]  max_features=auto, n_estimators=200, score=0.43119266055045874, total=   0.4s
[CV] max_features=auto, n_estimators=200 .............................


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.7s remaining:    0.0s


[CV]  max_features=auto, n_estimators=200, score=0.45161290322580644, total=   0.4s
[CV] max_features=auto, n_estimators=200 .............................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.2s remaining:    0.0s


[CV]  max_features=auto, n_estimators=200, score=0.4423963133640553, total=   0.4s
[CV] max_features=auto, n_estimators=700 .............................
[CV]  max_features=auto, n_estimators=700, score=0.3881278538812785, total=   1.6s
[CV] max_features=auto, n_estimators=700 .............................
[CV]  max_features=auto, n_estimators=700, score=0.43119266055045874, total=   1.6s
[CV] max_features=auto, n_estimators=700 .............................
[CV]  max_features=auto, n_estimators=700, score=0.4608294930875576, total=   1.7s
[CV] max_features=auto, n_estimators=700 .............................
[CV]  max_features=auto, n_estimators=700, score=0.45161290322580644, total=   1.5s
[CV] max_features=auto, n_estimators=700 .............................
[CV]  max_features=auto, n_estimators=700, score=0.4423963133640553, total=   1.5s
[CV] max_features=sqrt, n_estimators=200 .............................
[CV]  max_features=sqrt, n_estimators=200, score=0.3881278538812785, total

KeyboardInterrupt: 

In [30]:
model = RandomForestClassifier(max_features='log2',n_estimators=200)
run_experiment2(model, X, y)

Scores [0.6666666666666666, 0.6, 0.7, 0.7777777777777778, 0.5, 0.4444444444444444, 0.3, 0.1, 0.6, 0.2222222222222222, 0.4, 0.4, 0.3, 0.5, 0.1, 0.2222222222222222, 0.2, 0.4444444444444444, 0.4, 0.6, 0.3, 0.3333333333333333, 0.5, 0.3, 0.3333333333333333, 0.4, 0.3, 0.5555555555555556, 0.2, 0.5555555555555556, 0.3, 0.3, 0.6, 0.4444444444444444, 0.5, 0.7, 0.3, 0.8, 0.5, 0.6666666666666666, 0.5, 0.5555555555555556, 0.5, 0.7, 0.3333333333333333, 0.3333333333333333, 0.6, 0.3, 0.4]
Average score 0.4405895691609978
Total runs: 49


In [36]:
model = SVC(C=1, gamma=.1, kernel='rbf')
run_experiment2(model, X, y)

[0.66667, 0.5, 0.5, 0.55556, 0.5, 0.22222, 0.4, 0.3, 0.3, 0.22222, 0.2, 0.4, 0.5, 0.5, 0.5, 0.22222, 0.4, 0.55556, 0.3, 0.4, 0.3, 0.22222, 0.5, 0.3, 0.22222, 0.5, 0.3, 0.66667, 0.4, 0.66667, 0.3, 0.2, 0.5, 0.66667, 0.4, 0.3, 0.2, 0.7, 0.6, 0.66667, 0.6, 0.55556, 0.3, 0.4, 0.33333, 0.0, 0.4, 0.5, 0.5]
Average score 0.41519274376417237
Total runs: 49


In [ ]:
MLP_param_selection(X,y)

In [32]:
model = MLPClassifier(solver='lbfgs', alpha=10, max_iter=1500, hidden_layer_sizes=10)
run_experiment2(model, X, y)

Scores [0.6666666666666666, 0.5, 0.5, 0.5555555555555556, 0.5, 0.2222222222222222, 0.7, 0.3, 0.3, 0.2222222222222222, 0.6, 0.4, 0.5, 0.5, 0.5, 0.2222222222222222, 0.4, 0.5555555555555556, 0.3, 0.4, 0.3, 0.2222222222222222, 0.5, 0.3, 0.2222222222222222, 0.5, 0.3, 0.6666666666666666, 0.4, 0.6666666666666666, 0.3, 0.2, 0.5, 0.6666666666666666, 0.4, 0.3, 0.2, 0.7, 0.6, 0.6666666666666666, 0.6, 0.5555555555555556, 0.3, 0.5, 0.3333333333333333, 0.0, 0.4, 0.5, 0.5]
Average score 0.43151927437641735
Total runs: 49


In [ ]:
print(3)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
auto_y = df_out['Change SPY']

In [ ]:
auto_y

In [ ]:
plot_acf(auto_y, lags = 50)